In [2]:
import pandas as pd
import pickle

In [ ]:
file_path = '../Data/activePlayerCareerLogData.pickle'
PLAYER = 'kyrie irving'

In [ ]:
# Load collected player data from pickle file
with open(file_path, 'rb') as file:
    activePlayerCareerLog = pickle.load(file)

In [ ]:
# Clean Data and add features

for name, df_player in activePlayerCareerLog.items():
    df_player['WL'].replace({'L': 0, 'W': 1}, inplace=True)
    df_player['REST'] = 0
    
    for idx in df_player.index[1:]:
        date_difference = df_player['GAME_DATE'].iloc[idx] - df_player['GAME_DATE'].iloc[idx - 1]
        df_player.at[idx, 'REST'] = date_difference.days

    df_player.at[0, 'REST'] = 4 # assumed rest from NBA preseason to season start

### Time Series Feature Engineering

In [ ]:
target = 'PTS'
cols_to_drop = ['SEASON_ID', 'Player_ID', 'GAME_DATE', 'OPP', 'TEAM', target + '_target']

In [ ]:
# one lag time step
def shift_target(df):
    df[target + '_target'] = df[target].shift(-1)
    df.dropna(inplace=True)
    return df

In [ ]:
# rolling window
def recent_average(df, target, games=5):
    shifted_stat = df[target]
    window = shifted_stat.rolling(window=games).mean()
    df['LAST_' + str(games) + '_' + target] = window
    return df

def recent_percentage(df, target1, target2, games=5):
    shifted_stat1 = df[target1]
    shifted_stat2 = df[target2]
    stat_percentage = shifted_stat1 / shifted_stat2
    window = stat_percentage.rolling(window=games).mean()
    df['LAST_' + str(games) + '_' + target1 + '_PCT'] = window
    return df

In [ ]:
for player in activePlayerCareerLog.values():
    player = shift_target(player)
    player = recent_average(player, target, 3)
    player = recent_average(player, target, 5)
    player = recent_average(player, target, 7)
    player = recent_average(player, 'PLUS_MINUS', 3)
    player = recent_percentage(player, 'FGM', 'FGA', 5)